In [23]:
import numpy as np
import numba
from scipy.integrate import quad
from scipy.special import erf
from math import log, sqrt, exp, pi


def payoff_call(S_T:float, K:float):
    """
    Standard European call option payoff for the MLMC pricer

    :param S_T: Stock price
    :param K: Option price

    :return: payoff
    """
    return np.maximum(S_T - K, 0.0)

def smooth_payoff_call(S:np.ndarray, K:float, eps:float=0.01) ->np.ndarray:
    """
      Smoothed call payoff function
    :param S: Numpy of simulated stock prices
    :param K: Strike price
    :param eps: smoothness factor for payoff
    :return:
    """

    x = S - K
    s = eps * K
    z = x / (s * np.sqrt(2.0))

    # Clip to prevent overflow in the exp function
    z = np.clip(z, -10, 10)

    # CDF of SND
    Phi = 0.5 * (1.0 + erf(z))

    # PDF of SND
    phi = np.exp(-z**2) / np.sqrt(2.0 * np.pi)
    return x * Phi + s * phi


def heston_characteristic_function(u, S0, v0, r, T, kappa, theta, sigma_v, rho):
    """
    Calculates the Heston characteristic function, which is the Fourier transform of the
    probability density function of the log-asset price

    :param u: Frequency domain variable
    :param S0: Initial stock price
    :param v0: Initial variance
    :param r: Risk free rate
    :param T: Time to maturity
    :param kappa: Rate of Mean Reversion
    :param theta: Long term average variance
    :param sigma_v: Volatility of Volatility
    :param rho: Correlation coefficient

    :return: The complex value of the characteristic function
    """
    # Use 1j for clarity
    i = 1j

    #  xi and d are intermediate variables that appear throughout the solution d can be complex, so np.sqrt (which handles complex numbers) is used
    xi = kappa - rho * sigma_v * u * i
    d = np.sqrt(xi**2 + sigma_v**2 * (u * i + u**2))

    # The simple drift and initial price term
    A1 = u * i * (log(S0) + r * T)
    A2 = (kappa * theta / sigma_v**2) * (xi - d) * T

    #Avoid overflow in exponentials
    if np.real(d * T) > 50:
        # For very large d*T, use approximation
        A3 = -2 * (kappa * theta / sigma_v**2) * log(2) - 2 * (kappa * theta / sigma_v**2) * log(-d/2)
    else:
        g = (xi - d) / (xi + d)
        A3 = -2 * (kappa * theta / sigma_v**2) * np.log((1 - g * np.exp(-d * T)) / (1 - g))

    B = (v0 / sigma_v**2) * (xi - d) * (1 - np.exp(-d * T)) / (1 - g * np.exp(-d * T)) if np.abs(d * T) < 50 else 0

    # Combine all components and exponentiate to get the final characteristic function value
    return np.exp(A1 + A2 + A3 + B)


def heston_call_price_fourier(S0, K, r, T, v0, kappa, theta, sigma_v, rho):
    """
    Prices a European call option using the Heston model via Fourier technique

    :param S0: Stock price
    :param K: Strike price
    :param r: Risk free rate
    :param T: Time to maturity
    :param v0: Initial variance
    :param kappa: Rate of Mean Reversion
    :param theta: Long term average variance
    :param sigma_v: Volatility of Volatility
    :param rho: Correlation coefficient

    :return: The calculated price of the European call option
    """
    import cmath

    def integrand_P1(u):
        """
        First probability integrand
        """
        if u == 0:
            return 0.0
        try:
            # Ensure scalar
            u = float(u)

            # The characteristic function is evaluated at u-1j for P1
            phi = heston_characteristic_function(u - 1j, S0, v0, r, T, kappa, theta, sigma_v, rho)

            # The full integrand from the inversion formula
            result = (phi * cmath.exp(-1j * u * log(K)) / (1j * u))

            # Extract real part as float
            return float(result.real)
        except:
            return 0.0

    def integrand_P2(u):
        """Second probability integrand"""
        if u == 0:
            return 0.0
        try:

            u = float(u)
            # The characteristic function is evaluated at u for P2, which corresponds to the standard risk-neutral measure
            phi = heston_characteristic_function(u, S0, v0, r, T, kappa, theta, sigma_v, rho)
            result = (phi * cmath.exp(-1j * u * log(K)) / (1j * u))
            return float(result.real)
        except:
            return 0.0

    # Calculate the integrals I1 and I2 and discard error estimates
    try:
        I1, _ = quad(integrand_P1, 1e-8, 50, limit=200, epsabs=1e-12)
        I2, _ = quad(integrand_P2, 1e-8, 50, limit=200, epsabs=1e-12)

        # Recover the probabilities P1 and P2
        P1 = 0.5 + (1/pi) * I1
        P2 = 0.5 + (1/pi) * I2

        price = S0 * P1 - K * exp(-r * T) * P2
        return max(price, 0.0)
    except Exception as e:
        print(f"Fourier integration failed: {e}")

        from math import sqrt as math_sqrt

        # Simple approximation
        sigma_avg = math_sqrt(v0)
        d1 = (log(S0/K) + (r + 0.5*sigma_avg**2)*T) / (sigma_avg*math_sqrt(T))
        d2 = d1 - sigma_avg*math_sqrt(T)

        # Rough normal CDF approximation
        N_d1 = 0.5 * (1 + erf(d1/math_sqrt(2)))
        N_d2 = 0.5 * (1 + erf(d2/math_sqrt(2)))

        return max(S0 * N_d1 - K * exp(-r * T) * N_d2, 0.0)

@numba.jit(nopython=True)
def heston_euler_paths(S0:float, v0: float, r:float, T:float, kappa:float, theta:float, sigma_v:float, rho:float, n_steps:int, dW_S:np.ndarray, dW_v:np.ndarray) :
    """
    Simulates Heston paths using the Euler-Maruyama scheme

    :param S0: Initial stock price
    :param v0: Initial variance
    :param r: Risk free rate
    :param T: Time to maturity
    :param kappa: Rate of Mean Reversion
    :param theta: Long term average variance
    :param sigma_v: Volatility of Volatility
    :param rho: Correlation coefficient
    :param n_steps: Number of steps
    :param dW_S: Wiener increments for this stock
    :param dW_v: Wiener increments for the volatility

    :return: NumPy array of simulated stock prices at maturity
    """

    dt = T / n_steps
    X = np.full(dW_S.shape[0], log(S0), dtype=np.float64)
    v = np.full(dW_S.shape[0], v0, dtype=np.float64)
    for j in range(n_steps):
        # Ensure volatility is non negative
        v_pos = np.maximum(v, 0.0)
        sqrt_v_pos = np.sqrt(v_pos)

        # Update variance
        v = v + kappa * (theta - v_pos) * dt + sigma_v * sqrt_v_pos * dW_v[:, j]

        #Truncate volatility and update log price
        v = np.maximum(v, 0.0)
        X = X + (r - 0.5 * v_pos) * dt + sqrt_v_pos * dW_S[:, j]
    return np.exp(X)

@numba.jit(nopython=True)
def heston_milstein_paths(S0:float, v0: float, r:float, T:float, kappa:float, theta:float, sigma_v:float, rho:float, n_steps:int, dW_S:np.ndarray, dW_v:np.ndarray) :
    """
    Simulates Heston paths using the Milstein scheme directly on the price process

    :param S0: Initial stock price
    :param v0: Initial variance
    :param r: Risk free rate
    :param T: Time to maturity
    :param kappa: Rate of Mean Reversion
    :param theta: Long term average variance
    :param sigma_v: Volatility of Volatility
    :param rho: Correlation coefficient
    :param n_steps: Number of steps
    :param dW_S: Wiener increments for this stock
    :param dW_v: Wiener increments for the volatility

    :return: NumPy array of simulated stock prices at maturity
    """
    dt = T / n_steps
    S = np.full(dW_S.shape[0], S0, dtype=np.float64)
    v = np.full(dW_S.shape[0], v0, dtype=np.float64)
    for j in range(n_steps):
        v_pos = np.maximum(v, 0.0)
        sqrt_v_pos = np.sqrt(v_pos)
        # Variance update with Milstein term
        milstein_v = 0.25 * sigma_v**2 * (dW_v[:, j]**2 - dt)
        v = v + kappa * (theta - v_pos) * dt + sigma_v * sqrt_v_pos * dW_v[:, j] + milstein_v
        v = np.maximum(v, 0.0)
        # Stock price update with Milstein term
        milstein_S = 0.5 * v_pos * S * (dW_S[:, j]**2 - dt)
        S = S + r * S * dt + sqrt_v_pos * S * dW_S[:, j] + milstein_S
        S = np.maximum(S, 1e-12)
    return S

def make_ref_normals(n_paths:int, n_ref:int, seed=123, antithetic:bool=True):
    """
    Generate a set of independent standard normal distributions

    :param n_paths: Number of paths
    :param n_ref: Number of references
    :param seed: Random seed
    :param antithetic: Whether or not to use antithetic

    :return: 2d array of correlated Brownian motion
    """
    rng = np.random.default_rng(seed)
    Z1 = rng.normal(0.0, 1.0, size=(n_paths, n_ref))
    Z2 = rng.normal(0.0, 1.0, size=(n_paths, n_ref))
    if antithetic:
        Z1 = np.vstack([Z1, -Z1])
        Z2 = np.vstack([Z2, -Z2])
    return Z1, Z2


def block_normals(Z: np.ndarray, m:int):
    """
    Aggregate fine gird normal variables to create coarse ones

    :param Z: Array of fine grid normal variables
    :param m: Refinement factor

    :return: Array of blocked normal variables for coarse grid
    """
    N, N_ref = Z.shape
    return Z.reshape(N, N_ref // m, m).sum(axis=2) / np.sqrt(m)

def run_weak_convergence_analysis(scheme_name, path_simulator):
    """
    Runs a weak convergence test for a given Heston scheme

    :param scheme_name: Name of the scheme
    :param path_simulator: Heston path simulator function
    """
    print(f"\n--- Weak Convergence Analysis for {scheme_name} Scheme ---")
    steps_list = (16, 32, 64, 128, 256, 512)
    n_paths = 100_000
    n_ref = 2048

    # Generate a fixed set of random numbers for consistency
    Z1_ref, Z2_ref = make_ref_normals(n_paths, n_ref)

    dts, prices, errors = [], [], []
    ref_price = 0.0

    for i, n in enumerate(steps_list):
        dt = T / n

        Z1, Z2 = block_normals(Z1_ref, n_ref // n), block_normals(Z2_ref, n_ref // n)

        # Create correlated Wiener increments
        dW_v = Z1 * sqrt(dt)
        dW_S = (rho * Z1 + sqrt(1 - rho**2) * Z2) * sqrt(dt)

        S_T = path_simulator(S0, v0, r, T, kappa, theta, sigma_v, rho, n, dW_S, dW_v)

        # Use smooth payoff for weak order estimate
        price = exp(-r * T) * np.mean(smooth_payoff_call(S_T, K))
        dts.append(dt); prices.append(price)

        if i == len(steps_list) - 1:
            ref_price = price
        print(f"Steps: {n:4d}, dt: {dt:.5f}, Smoothed Price: {price:.5f}")

    errors = [abs(p - ref_price) for p in prices[:-1]]
    if len(errors) >= 3:
        # Use last few lines for a more stable estimate as initial one fuzzes results
        log_dts = np.log(np.array(dts[-4:-1]))
        log_errors = np.log(np.array(errors[-3:]))
        slope, _ = np.polyfit(log_dts, log_errors, 1)
    else:
        # Deafult to 0 if not enough data for a fit
        slope = 0.0

    print(f"\n Estimated Weak Order for {scheme_name}: {slope:.3f}")
    return slope

def mlmc_sample_level(l: int , N_l: int, path_simulator, rng, M=2):
    """Generic MLMC level sampler

    :param l: Current level index
    :param N_l: Number of sample to generate for the levele
    :param path_simulator: Heston path simulator function
    :param rng: Random number generator
    :param M: Refinement factor

    :return: Generated samples and computational cost
    """
    n_fine = M**l
    dt_f = T / n_fine
    cost = n_fine

    # Generate a single set of reference random numbers for this level
    Z1 = rng.normal(0.0, 1.0, size=(N_l, n_fine))
    Z2 = rng.normal(0.0, 1.0, size=(N_l, n_fine))
    dW_v_f = Z1 * sqrt(dt_f)
    dW_S_f = (rho * Z1 + sqrt(1 - rho**2) * Z2) * sqrt(dt_f)

    if l == 0:
        S_f = path_simulator(S0, v0, r, T, kappa, theta, sigma_v, rho, n_fine, dW_S_f, dW_v_f)
        Y = exp(-r * T) * payoff_call(S_f, K)

        # Create coarse increments by summing fine increments
    else:
        dW_v_c = dW_v_f.reshape(N_l, n_fine // M, M).sum(axis=2)
        dW_S_c = dW_S_f.reshape(N_l, n_fine // M, M).sum(axis=2)

        # Simulate fine and coarse paths
        S_f = path_simulator(S0, v0, r, T, kappa, theta, sigma_v, rho, n_fine, dW_S_f, dW_v_f)
        S_c = path_simulator(S0, v0, r, T, kappa, theta, sigma_v, rho, n_fine // M, dW_S_c, dW_v_c)

        # Estimator is discounted difference in payoffs
        Y = exp(-r * T) * (payoff_call(S_f, K) - payoff_call(S_c, K))

    return Y, cost

def mlmc_adaptive_pricer(scheme_name, path_simulator, eps=0.01, seed=42):
    """
    Generic adaptive MLMC pricer

    :param scheme_name: Name of the scheme
    :param path_simulator: Heston path simulator function
    :param eps: RMSE target
    :param seed: Random seed

    :return: final price and its standard error
    """

    rng = np.random.default_rng(seed)
    L, L_min, L_max, N0, M = 2, 2, 10, 2000, 2

    means = np.zeros(L_max + 1)
    vars = np.zeros(L_max + 1)
    Nl = np.zeros(L_max + 1, dtype=int)
    costs = np.zeros(L_max + 1)

    print(f"\n--- MLMC Pricer for {scheme_name} Scheme (Target SE: {eps:.4f}) ---")

    # Initial run for initial estimates
    for l in range(L + 1):
        Nl[l] = max(100, N0 // (M**l))
        Y, cost = mlmc_sample_level(l, Nl[l], path_simulator, rng, M)
        means[l] = np.mean(Y)
        vars[l] = np.var(Y, ddof=1)
        costs[l] = cost
        print(f"Initial Level {l}: N={Nl[l]}, mean={means[l]:+.4e}, var={vars[l]:.2e}")

    # Adaptive refinement loop
    for iter_count in range(20):
        total_var = sum(vars[l] / max(Nl[l], 1) for l in range(L + 1))

        # Check for convergence
        if total_var < eps**2:
            print(f"Converged: total variance {total_var:.3e} < target {eps**2:.3e}")
            break

        # Add new level if bias is too high
        if L < L_max and abs(means[L]) > eps / sqrt(2):
            L += 1
            Nl[L] = max(100, N0 // (M**L))
            Y, cost = mlmc_sample_level(L, Nl[L], path_simulator, rng, M)
            means[L] = np.mean(Y)
            vars[L] = np.var(Y, ddof=1)
            costs[L] = cost
            print(f"Added Level {L}: N={Nl[L]}, mean={means[L]:+.4e}, var={vars[L]:.2e}")

        # Recalculate optimal samples and add more
        N_opt = np.ceil(eps**-2 * np.sqrt(vars[:L+1] / costs[:L+1]) * sum(np.sqrt(vars[:L+1] * costs[:L+1])))
        for l in range(L + 1):
            extra = max(0, int(N_opt[l] - Nl[l]))
            if extra > 0:
                Y_ex, _ = mlmc_sample_level(l, extra, path_simulator, rng, M)

                # Update mean and variance with stable formulas
                total_N = Nl[l] + extra
                new_mean = (Nl[l] * means[l] + np.sum(Y_ex)) / total_N

                if Nl[l] > 1 and extra > 1:
                    vars[l] = ((Nl[l] - 1) * vars[l] + (extra - 1) * np.var(Y_ex, ddof=1) +
                               Nl[l] * (means[l] - new_mean)**2 + extra * (np.mean(Y_ex) - new_mean)**2) / (total_N - 1)
                means[l] = new_mean
                Nl[l] = total_N

    # Summary of levels
    print("\n--- Final MLMC Level Breakdown ---")
    print(f"{'Level':<5} | {'Nl':<10} | {'Mean':<12} | {'Variance':<12} | {'Cost':<10}")
    print("-" * 55)
    for l in range(L + 1):
        print(f"{l:<5} | {Nl[l]:<10} | {means[l]:<12.4e} | {vars[l]:<12.2e} | {Nl[l]*costs[l]:<10.0f}")

    price = sum(means[:L+1])
    se = sqrt(sum(vars[l] / max(Nl[l], 1) for l in range(L + 1)))
    print(f"\n Final Price for {scheme_name}: {price:.5f} (SE: {se:.5f})")
    return price, se


if __name__ == "__main__":
    schemes = {
        "Euler": heston_euler_paths,
        "Milstein": heston_milstein_paths,
    }

    S0, K, r, T = 1.0, 1.0, 0.05, 1.0
    v0, kappa, theta, sigma_v, rho = 0.04, 2.0, 0.04, 0.3, -0.7

    print("Fourier method for semi analytical price")
    fourier_price = heston_call_price_fourier(S0, K, r, T, v0, kappa, theta, sigma_v, rho)
    print(f"Heston Call Price (Fourier): {fourier_price:.6f}")
    print("=" * 60)


    # Weak Convergence Analysis
    weak_orders = {name: run_weak_convergence_analysis(name, sim) for name, sim in schemes.items()}
    prices = {name: mlmc_adaptive_pricer(name, sim) for name, sim in schemes.items()}

    print(f"{'Method':<15} | {'Weak Order':<12} | {'MLMC Price':<12} | {'Std. Error':<12} | {'Error vs Fourier':<15}")
    print("-" * 60)
    print("-" * 80)
    print(f"{'Fourier (Ref)':<15} | {'N/A':<12} | {fourier_price:<12.6f} | {'N/A':<12} | {'0.000000':<15}")


    for name in schemes:
        order = weak_orders[name]
        price, se = prices[name]
        norm_err = abs(price - fourier_price) / fourier_price
        print(f"{name:<15} | {order:<12.3f} | {price:<12.6f} | {se:<12.6f} | {norm_err:<16.6f}")
        ci_lower = price - 1.96 * se
        ci_upper = price + 1.96 * se
        ci_str = f"[{ci_lower:.6f}, {ci_upper:.6f}]"

        # Check if Fourier price is within the CI
        is_in_ci = "Yes" if ci_lower <= fourier_price <= ci_upper else "No"
        print(f"{name:<15} | {order:<12.3f} | {price:<12.6f} | {ci_str:<25} | {is_in_ci:<8} | {norm_err:<15.6f}")

    print(f"{'='*80}")



Fourier method for semi analytical price
Heston Call Price (Fourier): 0.113779

--- Weak Convergence Analysis for Euler Scheme ---
Steps:   16, dt: 0.06250, Smoothed Price: 0.10379
Steps:   32, dt: 0.03125, Smoothed Price: 0.10379
Steps:   64, dt: 0.01562, Smoothed Price: 0.10379
Steps:  128, dt: 0.00781, Smoothed Price: 0.10377
Steps:  256, dt: 0.00391, Smoothed Price: 0.10374
Steps:  512, dt: 0.00195, Smoothed Price: 0.10376

 Estimated Weak Order for Euler: 0.882

--- Weak Convergence Analysis for Milstein Scheme ---
Steps:   16, dt: 0.06250, Smoothed Price: 0.10325
Steps:   32, dt: 0.03125, Smoothed Price: 0.10352
Steps:   64, dt: 0.01562, Smoothed Price: 0.10367
Steps:  128, dt: 0.00781, Smoothed Price: 0.10371
Steps:  256, dt: 0.00391, Smoothed Price: 0.10372
Steps:  512, dt: 0.00195, Smoothed Price: 0.10374

 Estimated Weak Order for Milstein: 0.767

--- MLMC Pricer for Euler Scheme (Target SE: 0.0100) ---
Initial Level 0: N=2000, mean=+1.1031e-01, var=2.41e-02
Initial Level 1: 